<a href=\"https://colab.research.google.com/github/Samir-atra/CancerDetector/blob/main/Cancer_Detector/Transfer.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>

In [ ]:
# 1. Imports and Setup
# --------------------
# This cell installs and imports the necessary libraries for the project.

!pip install keras_tuner -q
import tensorflow as tf
import keras_tuner
import numpy as np
import os
import pathlib
import matplotlib.pyplot as plt
import datetime
import tensorboard
import IPython
import sklearn
import cv2
import sys
import time

In [ ]:
# 2. Mount Google Drive
# ---------------------
# Mount Google Drive to access the dataset, which is stored there.
# This is necessary when running the notebook in Google Colab.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive\n

In [ ]:
# 3. Dataset Loading
# ------------------
# Load the training, validation, and testing datasets from Google Drive.

data_path = pathlib.Path('/content/drive/MyDrive/archiveX3/Training/')
data_path_test = pathlib.Path('/content/drive/MyDrive/archiveX3/Testing/')
AUTOTUNE = tf.data.AUTOTUNE

start = time.time()
# Create the training and validation datasets.
dataset_path, dataset_path_val = tf.keras.utils.image_dataset_from_directory(
    data_path,
    labels='inferred',
    validation_split=0.2,
    subset='both',
    seed=1,
    batch_size=5,
    image_size=(299, 299), # Image size must match InceptionV3 input.
    color_mode="rgb",
    shuffle=True)

# Create the testing dataset.
dataset_path_test = tf.keras.utils.image_dataset_from_directory(
    data_path_test,
    labels='inferred',
    seed=3,
    batch_size=5,
    image_size=(299, 299),
    color_mode="rgb",
    shuffle=True)

end = time.time()
print(f"Time elapsed for data loading: {end - start:.2f} seconds")

Found 8582 files belonging to 4 classes.\nUsing 6866 files for training.\nFound 8582 files belonging to 4 classes.\nUsing 1716 files for validation.\nFound 1705 files belonging to 4 classes.\ntime elapsed =  32.22160363197327\n

In [ ]:
# 4. Performance Optimization (Commented Out)
# -------------------------------------------
# Caching and prefetching can be enabled for faster training.

# AUTOTUNE = tf.data.AUTOTUNE
# dataset_path = dataset_path.cache().prefetch(buffer_size=AUTOTUNE)
# dataset_path_val = dataset_path_val.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# 5. Load Base Model
# ------------------
# Download the InceptionV3 model, pre-trained on ImageNet.
# `include_top=False` removes the final classification layer.

base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=(299, 299, 3),
    include_top=False,
    weights="imagenet")

In [ ]:
# 6. Save Base Model
# ------------------
# Save the downloaded base model to avoid re-downloading it every time.

saving_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedBaseModel.h5')
base_model.save(saving_path)

In [ ]:
# 7. Transfer Learning and Fine-Tuning
# --------------------------------------
# This cell implements the core logic for transfer learning.
# The process involves two main stages:
#   1. Feature Extraction: Train only the new classification head on top of the frozen base model.
#   2. Fine-Tuning: Unfreeze some layers of the base model and train the entire model with a very low learning rate.

# Load the saved base model.
model_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedBaseModel.h5')
base_model = tf.keras.models.load_model(model_path)

# --- Stage 1: Feature Extraction ---
base_model.trainable = False # Freeze the base model.

# Define preprocessing and augmentation layers.
rescaling = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1) # Rescale to [-1, 1] for InceptionV3.
])
augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1)
])

# Build the final model by adding a new classification head.
inputs = tf.keras.Input(shape=(299, 299, 3))
x = augmentation(inputs)
x = rescaling(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

# The following section for hyperparameter tuning with Keras Tuner is commented out.
# It seems the author has already found a good learning rate.
# def build_model(hp):
#     ...
#     return model
# tuner = keras_tuner.RandomSearch(...)
# tuner.search(...)

# Compile the model for the feature extraction phase.
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005670323757088328),
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly=True)

# Train the model (only the new head).
model.fit(
    dataset_path,
    epochs=15,
    validation_data=dataset_path_val)

# Evaluate the model after the first stage.
model.evaluate(dataset_path_test, batch_size=5, verbose=1)
model.summary()

# --- Stage 2: Fine-Tuning (Commented Out) ---
# In this stage, the base model is unfrozen to train the entire network.

base_model.trainable = True

# It's important to freeze BatchNormalization layers when fine-tuning.
for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

# Re-compile the model with a very low learning rate for fine-tuning.
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001),
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly=True)

# The fine-tuning training is commented out, but would typically be run here.
# model.fit(
#     dataset_path,
#     epochs=30,
#     initial_epoch=15, # Continue from the previous training.
#     validation_data=dataset_path_val)

# Evaluate the model after fine-tuning.
model.evaluate(dataset_path_test, batch_size=5, verbose=1)
model.summary()

# The code for saving the final fine-tuned model is also commented out.
# saving_path = pathlib.Path('/content/drive/MyDrive/archiveX3/SavedModelWithFT/')
# tf.keras.models.save_model(model, saving_path, ...)


Epoch 1/15\n1374/1374 [==============================] - 2010s 1s/step - loss: 0.5756 - accuracy: 0.7830 - val_loss: 0.4003 - val_accuracy: 0.8625\nEpoch 2/15\n1374/1374 [==============================] - 486s 354ms/step - loss: 0.3808 - accuracy: 0.8602 - val_loss: 0.3462 - val_accuracy: 0.8759\nEpoch 3/15\n1374/1374 [==============================] - 488s 355ms/step - loss: 0.3311 - accuracy: 0.8793 - val_loss: 0.2980 - val_accuracy: 0.8887\nEpoch 4/15\n1374/1374 [==============================] - 493s 359ms/step - loss: 0.3019 - accuracy: 0.8865 - val_loss: 0.2656 - val_accuracy: 0.9091\nEpoch 5/15\n1374/1374 [==============================] - 493s 359ms/step - loss: 0.2756 - accuracy: 0.9002 - val_loss: 0.2547 - val_accuracy: 0.9108\nEpoch 6/15\n1374/1374 [==============================] - 489s 356ms/step - loss: 0.2570 - accuracy: 0.9049 - val_loss: 0.2322 - val_accuracy: 0.9184\nEpoch 7/15\n1374/1374 [==============================] - 489s 356ms/step - loss: 0.2499 - accuracy: 0.